In [1]:
import os, sys, json, torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

# 경로 설정
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from multilayer_model.m3_hybrid_final import HierarchicalHybridCVAE3
from multilayer_loss.l_multi3_hybrid_final import l_multi3_hybrid_final
from vae_earlystopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_runs = 20
total_epochs = 800
r2_results = []

In [2]:
def train_single_model(model, optimizer, train_loader, val_loader, mode='bce'):
    es = EarlyStopping(patience=40, min_delta=1e-9)
    for epoch in range(1, total_epochs + 1):
        model.train()
        train_loss = 0
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            optimizer.zero_grad()
            
            out = model(b1, b2, b3, bc)
            loss_res = l_multi3_hybrid_final(out['recons'], out['logits'], [b1, b2, b3], out['mus'], out['lvs'], epoch, total_epochs)
            
            # 모드에 따라 주 손실 선택 (BCE 또는 MSE)
            # 계층별 KL Balancing은 내부 l_multi3_hybrid_final에서 자동 적용됨
            main_loss = loss_res['bce'] if mode == 'bce' else loss_res['mse']
            total_loss = main_loss + loss_res['kl']
            
            total_loss.backward()
            optimizer.step()
            train_loss += total_loss.item()

        # Validation
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vout = model(v1, v2, v3, vc)
                v_res = l_multi3_hybrid_final(vout['recons'], vout['logits'], [v1, v2, v3], vout['mus'], vout['lvs'], epoch, total_epochs)
                v_loss += (v_res['bce'] if mode == 'bce' else v_res['mse']).item()
        
        avg_v_loss = v_loss / len(val_loader)
        if epoch % 100 == 0: 
            print(f"  [Epoch {epoch}] Mode: {mode.upper()} | Val Loss: {avg_v_loss:.6f}")
        
        if es(avg_v_loss, model):
            print(f"  Early stopping at epoch {epoch}")
            break
    es.load_best_model(model)

In [3]:
print(f"Starting {num_runs} independent runs on {device}...")

# 1. 데이터 로드
x1_raw = np.load('../data/metal.npy')
x2_raw = np.load('../data/support_norm.npy')
x3_raw = np.load('../data/pre_fin.npy')
c_raw = np.load('../data/re_fin.npy')

for run in range(1, num_runs + 1):
    print(f"\n▶ RUN {run}/{num_runs} Processing...")
    
    # 데이터 분할 (Run별 시드 변경)
    idx = np.arange(len(x1_raw))
    tr_idx, te_idx = train_test_split(idx, test_size=0.2, random_state=run)
    tr_idx, va_idx = train_test_split(tr_idx, test_size=0.2, random_state=run)

    # 스케일링
    sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
    
    def prep(data, idx, sc, fit=False):
        d = data[idx]
        return sc.fit_transform(d) if fit else sc.transform(d)

    x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
    x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
    x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
    c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

    def to_t(a): return torch.tensor(a, dtype=torch.float32)
    train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
    test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

    # 모델 초기화
    x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]
    c_dim = c_tr.shape[1]
    
    model_bce = HierarchicalHybridCVAE3(x_dims, c_dim, [16, 8, 4]).to(device)
    model_mse = HierarchicalHybridCVAE3(x_dims, c_dim, [16, 8, 4]).to(device)
    
    opt_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
    opt_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)

    # 1. Train BCE (Classification focused)
    train_single_model(model_bce, opt_bce, train_loader, val_loader, mode='bce')
    # 2. Train MSE (Regression focused)
    train_single_model(model_mse, opt_mse, train_loader, val_loader, mode='mse')

    # 3. Combined Evaluation (Support Phase focus)
    model_bce.eval(); model_mse.eval()
    all_pred, all_true = [], []
    with torch.no_grad():
        for b1, b2, b3, bc in test_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            
            # BCE Prob (Presence)
            b_out = model_bce(b1, b2, b3, bc)
            prob2 = torch.sigmoid(b_out['logits'][1]) # Support Logit
            
            # MSE Value (Amount)
            m_out = model_mse(b1, b2, b3, bc)
            val2 = m_out['recons'][1] # Support Recons
            
            # 결합 및 역스케일링
            combined = (prob2 * val2).cpu().numpy()
            pred_inv = sc2.inverse_transform(combined)
            true_inv = sc2.inverse_transform(b2.cpu().numpy())
            
            all_pred.append(pred_inv); all_true.append(true_inv)

    y_true, y_pred = np.concatenate(all_true).flatten(), np.concatenate(all_pred).flatten()
    r2 = r2_score(y_true, y_pred)
    r2_results.append(r2)
    print(f"  → RUN {run} R2 Score: {r2:.6f}")

# 결과 요약 및 저장
final_stats = {
    "mean_r2": float(np.mean(r2_results)),
    "std_r2": float(np.std(r2_results)),
    "all_r2": [float(x) for x in r2_results]
}

with open('../model/final_runs_statistics.json', 'w') as f:
    json.dump(final_stats, f, indent=4)

print("\n" + "="*40)
print(f"🏁 20-RUNS FINAL STATISTICS")
print(f"Mean R2: {final_stats['mean_r2']:.6f} ± {final_stats['std_r2']:.6f}")
print("="*40)

Starting 20 independent runs on cuda...

▶ RUN 1/20 Processing...
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 7 out of 40
EarlyStopping counter: 8 out of 40
EarlyStopping counter: 9 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2